In [2]:
import os
from r2r import R2RClient
from scraper import Scraper
from splitter import Splitter
from io_helper import IOHelper
from server import ServerHelper
from prompt import PromptHelper

client = R2RClient(os.getenv("R2R_HOSTNAME", "http://localhost:7272"))
scraper = Scraper()
splitter = Splitter()
io_helper = IOHelper()
server = ServerHelper(client)
prompt = PromptHelper(client)

2024-10-21 20:04:54,612 - WARNING - langchain_community.utils.user_agent - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from r2r import R2RConfig

config = R2RConfig.from_toml("config.toml")

In [19]:
from r2r import R2RClient

client = R2RClient('http://localhost:7272')

In [33]:
client.user()['results']

from r2r import R2RClient
client = R2RClient('http://localhost:7272')

client.
user = client.register("random@random.com", "sdadsa")
user.get('results')

R2RException: {"detail":{"message":"User with this email already exists","error_type":"R2RException"}}

In [10]:
client.

{'results': [{'collection_id': '122fdf6a-e116-546b-a8f6-e4cb2e2c0a09',
   'name': 'Default',
   'description': 'Your default collection.',
   'created_at': '2024-10-21T14:00:56.412981Z',
   'updated_at': '2024-10-21T14:00:56.412981Z'}],
 'total_entries': 1}

In [20]:
import yaml
from pathlib import Path

def load_prompt(directory_path):
    if not directory_path:
        raise ValueError("No directory path provided")
    
    if not directory_path.is_dir():
        error_msg = f"The specified path is not a directory: {directory_path}"
        raise ValueError(error_msg)

    for yaml_file in directory_path.glob("*.yaml"):
        try:
            with open(yaml_file, "r") as file:
                data = yaml.safe_load(file)
                if not data:
                    print(f"Warning: YAML file {yaml_file} is empty")
                    continue
                if not isinstance(data, dict):
                    raise ValueError(f"Invalid format in YAML file {yaml_file}")
                
                for name, prompt_data in data.items():
                    print(f'{name}, {prompt_data["template"]}, {prompt_data.get("input_types", {})}')
                    
        except yaml.YAMLError as e:
            error_msg = f"Error loading prompts from YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)
        except KeyError as e:
            error_msg = f"Missing key in YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)
        except Exception as e:
            error_msg = f"Unexpected error loading YAML file {yaml_file}: {e}"
            raise ValueError(error_msg)

In [21]:
load_prompt(Path("my_templates"))

custom_rag_template, ## Task: You are an AI assistant specialized in providing accurate and context-based responses. Your task is to answer user queries using only the information from the provided context. Your responses should be factual, comprehensive, and directly reference the given context.
## Context: {context}
## Query: {query}
## Instructions: 1. Analyze the query and the provided context carefully. 2. Formulate a response based solely on the information in the context. 3. Use specific references (e.g., [1], [2]) to cite relevant parts of the context in your response. 4. Ensure your answer is as comprehensive as possible within the constraints of the given context. 5. If the query cannot be fully answered from the context:
  a. Provide partial information if available, clearly stating what aspects you can address.
  b. Indicate which parts of the query cannot be answered based on the given context.
6. If the query is entirely outside the scope of the provided context, state: "

In [28]:
files = list(io_helper.iterate_over_files("files"))
ingestion_resp = server.ingest_files(files)

Ingested: files\03 Kubernetes.pdf ...


[{'id': 'c1d89317-1011-5618-8754-4eafb384bd83',
  'title': '03 Kubernetes.pdf',
  'user_id': '2acb499e-8428-543b-bd85-0d9098718220',
  'type': 'pdf',
  'created_at': '2024-10-20T21:52:51.967235Z',
  'updated_at': '2024-10-20T21:52:51.967239Z',
  'ingestion_status': 'success',
  'kg_extraction_status': 'pending',
  'version': 'v0',
  'collection_ids': ['122fdf6a-e116-546b-a8f6-e4cb2e2c0a09'],
  'metadata': {'version': 'v0'}}]

In [13]:
chunks = server.document_chunks('c1d89317-1011-5618-8754-4eafb384bd83')

In [11]:
print(rag_rsp['completion']['choices'][0]['message']['content'])

Based on the provided text, here's how you can create a deployment with 5 replicas that is vertically and horizontally scalable and also create a LoadBalancer:

**Step 1: Create a Deployment**

To create a deployment with 5 replicas, you can use the following command:
```bash
docker service create --replicas=5 my-service
```
This will create a new service named `my-service` with 5 replicas.

**Step 2: Make the Service Vertically and Horizontally Scalable**

To make the service vertically scalable (i.e., increase resources for each replica), you can use the following command:
```bash
docker service update --resource=cpu=1000m my-service
```
This will set the CPU resource for each replica to 1 core.

To make the service horizontally scalable, you can use a load balancer. In Docker Swarm, you can create a LoadBalancer using the following command:
```bash
docker service create --name my-load-balancer --publish=8080:80 my-service
```
This will create a new service named `my-load-balancer` t

In [23]:
import os
import requests

URL = os.getenv('R2R_HOSTNAME', 'http://localhost:7272')

# 1. First get a token
LOGIN_SUFFIX = "/v2/login"

body = {
    "username": "",
    "password": ""
}

# https://r2r-docs.sciphi.ai/api-reference/endpoint/login
token_rsp = requests.post(f'{URL}{TOKEN_SUFFIX}', json="Refresh token")

if token_rsp.status_code == 200:
    TOKEN = token_rsp['results']['access_token']['token']
    
    # INDEX_SUFFIX = "/v2/create_vector_index"
    
    # # https://r2r-docs.sciphi.ai/api-reference/endpoint/create_vector_index
    # body = {
    #     "table_name": "CHUNKS",
    #     "index_method": "hnsw",
    #     "measure": "cosine_distance",
    #     "replace": False,
    #     "concurrently": True
    # }

    # headers = {
    #     "Content-Type": "application/json",
    #     "Authorization": f"Bearer {TOKEN}"
    # }

    # resp = requests.post(f"{URL}{INDEX_SUFFIX}", json=body, headers=headers)
    # print(f"Response code: {resp.status_code} - {resp.text}")
else:
    print(f'Something went wrong! {token_rsp.text}')

Something went wrong! {"detail":{"message":"Invalid token","error_type":"R2RException"}}
